In [1]:
import numpy as np
import xarray as xr

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


https://mediatum.ub.tum.de/1524895

Download the `5.625deg/geopotential_500` folder from the following link and place it in the `data` directory.

https://dataserv.ub.tum.de/index.php/s/m1524895?path=%2F5.625deg%2Fgeopotential_500

In [2]:
z500 = xr.open_mfdataset(f'data/geopotential_500/*.nc', combine='by_coords').z

In [3]:
z500

<xarray.DataArray 'z' (time: 350640, lat: 32, lon: 64)>
dask.array<concatenate, shape=(350640, 32, 64), dtype=float32, chunksize=(8784, 32, 64), chunktype=numpy.ndarray>
Coordinates:
    level    int32 500
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [4]:
data_train = z500.sel(time=slice('2016', '2017'))
data_test = z500.sel(time='2018')

In [5]:
data_train

<xarray.DataArray 'z' (time: 17544, lat: 32, lon: 64)>
dask.array<getitem, shape=(17544, 32, 64), dtype=float32, chunksize=(8784, 32, 64), chunktype=numpy.ndarray>
Coordinates:
    level    int32 500
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 2016-01-01 ... 2017-12-31T23:00:00
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [6]:
data_test

<xarray.DataArray 'z' (time: 8760, lat: 32, lon: 64)>
dask.array<getitem, shape=(8760, 32, 64), dtype=float32, chunksize=(8760, 32, 64), chunktype=numpy.ndarray>
Coordinates:
    level    int32 500
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 2018-01-01 ... 2018-12-31T23:00:00
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [7]:
data_mean = data_train.isel(time=slice(0, None, 10000)).mean().load()
data_std = data_train.isel(time=slice(0, None, 10000)).std().load()

In [8]:
# Normalize datasets
data_train = (data_train - data_mean) / data_std
data_test = (data_test - data_mean) / data_std

In [9]:
_, nlat, nlon = data_train.shape; nlat, nlon

(32, 64)

In [10]:
def create_training_data(da, lead_time_h, return_valid_time=False):
    """Function to split input and output by lead time."""
    X = da.isel(time=slice(0, -lead_time_h))
    y = da.isel(time=slice(lead_time_h, None))
    valid_time = y.time
    if return_valid_time:
        return X.values.reshape(-1, nlat*nlon), y.values.reshape(-1, nlat*nlon), valid_time
    else:
        return X.values.reshape(-1, nlat*nlon), y.values.reshape(-1, nlat*nlon)

In [11]:
lead_time_h = 1 * 24
X_train, y_train = create_training_data(data_train, lead_time_h)
X_test, y_test, valid_time = create_training_data(data_test, lead_time_h, return_valid_time=True)

In [12]:
data_mean.values

array(53908.11, dtype=float32)

In [13]:
X_train.shape

(17520, 2048)

In [14]:
X_test.shape

(8736, 2048)

In [15]:
np.savez('train_data.npz', X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, data_mean=data_mean.values, data_std=data_std.values)